In [19]:
import numpy as np
import time
from matplotlib import pyplot as plt
import NEON_AOP_waveform_tools #importing the functions
import sys

In [25]:
#define the PLS file fo the pulswaves data
plsFile = './2015032618_Q780_150326_194954_1_originalpoints.pls'

#set the first and last numbers in the range - in the bash script set the beginning and end
#call the script using this: python [name_of_script].py begin end
#begin = int(sys.argv[1])
#end = int(sys.arv[2])

begin = 0
end = 10


GaussianCalcTime = time.clock()
for pulse in range(begin,end):
    pulse_number = [pulse]
    coord_info, wave_in, wave_out = NEON_AOP_waveform_tools.waveform_read(pulse_number, plsFile)
    #print(coord_info) 
    #get the initial parameters
    initial_params_out = NEON_AOP_waveform_tools.get_params(wave_out[0])
    initial_params_in = NEON_AOP_waveform_tools.get_params(wave_in[0])
    
    num_peaks_in = np.array(initial_params_in[0,-1]) #the last one is the number of peaks
    WFtime = np.arange(1,244)
    temp_in_params = NEON_AOP_waveform_tools.get_gaussian_params(initial_params_in, wave_in[0],10,20,0)[0]
    temp_out_params = NEON_AOP_waveform_tools.get_gaussian_params(initial_params_out, wave_out[0],10,20,0)[0]

    fit_wave_final = NEON_AOP_waveform_tools.get_fit_wave(temp_in_params, 244, num_peaks_in)
    FWHM = (2.355*(temp_in_params[0,2:int(num_peaks_in*3):3]/2**0.5))/2
    
    trig_loc = temp_in_params[0,1:int(num_peaks_in*3):3]-FWHM-temp_out_params[0,1] #subtracting the outgoing pulse, need to normalize trigger locations by the locations of the outgoing pulse
    trig_loc = np.reshape(trig_loc,(int(num_peaks_in),1))

    pulse_coords = (np.matlib.repmat(trig_loc[:,0],3,1)).transpose()*coord_info[:,1:4]+coord_info[:,4:7]
    #print(pulse_coords)

print("total time for gaussian decomposition is {}".format(time.clock()-GaussianCalcTime))



read 0 pulses in file in 7.399999999790907e-05 seconds, we are [0.]% done
1
1
1
1
total time for gaussian decomposition is 7.347378000000006


In [26]:
GaussianCalcTime = time.clock()
for pulse in range(begin,end):
    pulse_number = [pulse]
    coord_info, wave_in, wave_out = NEON_AOP_waveform_tools.waveform_read(pulse_number, plsFile)

print("total time for gaussian decomposition is {}".format(time.clock()-GaussianCalcTime))


read 0 pulses in file in 7.700000000454565e-05 seconds, we are [0.]% done
total time for gaussian decomposition is 6.9853980000000035


In [20]:
%bash

#!/bin/bash -l
#$ -V
#$ -l h_rt=10:00:00
#$ -j y
#$ -pe omp 8
#$ -l mem_per_core=8G
#$ -N waveform_lidar_processing
#$ -m a

echo $1
echo $2
python [nameofscript.py] $1 $2


SyntaxError: invalid syntax (<ipython-input-20-fa67c88cb831>, line 11)

In [27]:
%R
#in R: paste this:
pulse_number = 20000000

sequence = seq(0,pulse_number, 10000)

for(i in 1:(length(sequence)-1)){
  begin = as.integer(sequence[i]+1)
  end = as.integer(sequence[i+1])
  print(begin)
  print(end)
  system(paste('qsub [script_path]', begin, end, sep = " "))
}


SyntaxError: invalid syntax (<ipython-input-27-4b2c50d1755c>, line 7)

In [ ]:
|